# Bank Marketing (with social/economic context)

It is a dataset that describing Portugal bank marketing campaigns results.
Conducted campaigns were based mostly on direct phone calls, offering bank client to place a term deposit.
If after all marking afforts client had agreed to place deposit - target variable (_y_) marked 'yes', otherwise 'no'

Downloaded from https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset


In [23]:
import pandas as pd

In [24]:
 df = pd.read_csv("../data/raw/bank-additional-full.csv", sep = ';')

In [25]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# Clean the dataset

### Duplicates

In [26]:
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print('There were {} duplicate rows!'.format(before - after))

There were 12 duplicate rows!


### Missing values

In [27]:
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

 ### Check if there are obviously incosistent data

In [28]:
for column in df.columns:
    unique_vals = df[column].unique()
    print('''{}: {}
_
    '''.format(column, unique_vals))


age: [56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89]
_
    
job: ['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
_
    
marital: ['married' 'single' 'divorced' 'unknown']
_
    
education: ['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
_
    
default: ['no' 'unknown' 'yes']
_
    
housing: ['no' 'yes' 'unknown']
_
    
loan: ['no' 'yes' 'unknown']
_
    
contact: ['telephone' 'cellular']
_
    
month: ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
_
    
day_of_week: ['mon' 'tue' 'wed' 'thu' 'fri']
_
    
duration: [ 261  149  226 ... 1246 1556 1868]
_
    
campaign: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 19 18 23 1

### Inconsistency between the columns *pdays* and *previous*. 
There are values of *previous* greater then zero even if *pdays* in the same row is equal to *999*.

Recap that
- *pdays* means _number of days that passed by after the client was last contacted from a previous campaign (numeric; ***999 means client was not previously contacted***)_
- *previous* means _number of contacts performed before this campaign and for this client_

In [29]:
df[df.pdays == 999].previous.value_counts()

0    35551
1     3696
2      349
3       50
4       12
5        2
6        1
Name: previous, dtype: int64

**Possible explanations**:
- there is division between contacts from a previous **campaign** and all other previous contacts
- this is a mistake

Relying on the statement *999 means client was not previously contacted* the second explanation is taken into account and the questionable values are set to *0*.

In [30]:
previous_mistake = df[df.pdays == 999].previous.index
df.loc[previous_mistake, 'previous'] = 0
df[df.pdays == 999].previous.value_counts()

0    39661
Name: previous, dtype: int64

### Illiterate education
Note: It is strange that people which are illiterate  have job positions like _admin_ or _enterpreneur_. But theoretically it is possible. Originally, it was intended to replace these values by *unknown*, but currently it is decided do not change it.

In [31]:
df[df.education == 'illiterate'].job.unique()
#illiterate = df[df.education == 'illiterate'].index
#df.loc[illiterate, 'education'] = 'unknown'
#df.education[illiterate].value_counts()

array(['blue-collar', 'admin.', 'housemaid', 'self-employed',
       'entrepreneur', 'retired'], dtype=object)

In [32]:
df.to_csv('../data/clean/bank-additional-full_cleaned.csv', index=False)